In [2]:
import plotly
import pandas as pd
import numpy as np
from pathlib import Path
import os

In [3]:
path_to_download_folder = str(os.path.join(Path.home(), "Downloads"))

In [4]:
admit = pd.read_csv(os.path.join(path_to_download_folder, "Admit.csv"))

In [5]:
admit.head()

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
0,21,22,165315,2196-04-09 12:26:00.000,2196-04-10 15:54:00.000,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00.000,2196-04-09 13:24:00.000,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00.000,2153-09-08 19:10:00.000,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00.000,2157-10-25 14:00:00.000,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00.000,2139-06-09 12:48:00.000,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00.000,2160-11-05 14:55:00.000,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00.000,2160-11-02 04:27:00.000,ACUTE CORONARY SYNDROME,0,1


Check to find the top 10 most common reasons for ER admission.

In [6]:
admits = admit['diagnosis'].value_counts()

In [7]:
admits.head(10)

NEWBORN                                                      7823
PNEUMONIA                                                    1566
SEPSIS                                                       1184
CONGESTIVE HEART FAILURE                                      928
CORONARY ARTERY DISEASE                                       840
CHEST PAIN                                                    778
INTRACRANIAL HEMORRHAGE                                       713
ALTERED MENTAL STATUS                                         712
GASTROINTESTINAL BLEED                                        686
CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS GRAFT /SDA     583
Name: diagnosis, dtype: int64

In [8]:
admit['ethnicity'].value_counts()

WHITE                                                       40996
BLACK/AFRICAN AMERICAN                                       5440
UNKNOWN/NOT SPECIFIED                                        4523
HISPANIC OR LATINO                                           1696
OTHER                                                        1512
ASIAN                                                        1509
UNABLE TO OBTAIN                                              814
PATIENT DECLINED TO ANSWER                                    559
ASIAN - CHINESE                                               277
HISPANIC/LATINO - PUERTO RICAN                                232
BLACK/CAPE VERDEAN                                            200
WHITE - RUSSIAN                                               164
MULTI RACE ETHNICITY                                          130
BLACK/HAITIAN                                                 101
ASIAN - ASIAN INDIAN                                           85
WHITE - OT

Take a look at the top admit reasons for each race.

In [9]:
def ethnicity_admit(admit, ethnicity):
    ethnicity = admit[admit['ethnicity'] ==  ethnicity].groupby(['diagnosis']).size()
    ethnicity = ethnicity.reset_index()
    ethnicity["Percentage"] = ethnicity[0]/ethnicity[0].sum()
    ethnicity = ethnicity.sort_values(by = "Percentage", ascending= False).head(5)
    
    return(ethnicity)


In [10]:
ethnicity_admit(admit, 'WHITE')

,diagnosis,0,Percentage
7683,NEWBORN,4755,0.116046
8365,PNEUMONIA,1166,0.028456
10149,SEPSIS,864,0.021086
3113,CONGESTIVE HEART FAILURE,614,0.014985
2522,CHEST PAIN,575,0.014033


In [11]:
ethnicity_admit(admit, "BLACK/AFRICAN AMERICAN")

,diagnosis,0,Percentage
1151,NEWBORN,865,0.159037
419,CONGESTIVE HEART FAILURE,176,0.032359
499,DIABETIC KETOACIDOSIS,165,0.030336
1234,PNEUMONIA,164,0.030153
1504,SEPSIS,137,0.025188


In [12]:
ethnicity_admit(admit, "ASIAN")

,diagnosis,0,Percentage
318,NEWBORN,715,0.474453
339,PNEUMONIA,38,0.025216
217,HYPERBILIRUBINEMIA,31,0.020571
408,SEPSIS,19,0.012608
247,INTRACRANIAL HEMORRHAGE,15,0.009954


In [13]:
ethnicity_admit(admit, "UNKNOWN/NOT SPECIFIED")

,diagnosis,0,Percentage
1183,NEWBORN,437,0.096639
555,CORONARY ARTERY DISEASE,185,0.040911
625,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,115,0.025431
433,CHEST PAIN\CATH,71,0.015701
616,CORONARY ARTERY DISEASE\CATH,68,0.015038


Looking at the top 20 diseases by length of stay (in days).

In [14]:
admit['staylength']= pd.to_datetime(admit['dischtime']) - pd.to_datetime(admit['admittime'])

In [15]:
admit['staylength'] = pd.to_timedelta(admit.staylength).dt.total_seconds()
# Convert seconds to days
admit['staylength'] = admit['staylength']/(24*60*60)

In [16]:
average_stay = admit[['staylength', 'diagnosis']].groupby(['diagnosis']).mean()
average_stay.sort_values(by = 'staylength', ascending  = False).head(20)

,staylength
diagnosis,
CROHN'S DISEASE;ABDOMINAL FISTULA,294.660417
ASPIRATION; FAILURE TO THRIVE,191.422917
CHRONIC LYMPHOCYTIC LEUKEMIA\BONE MARROW TRANSPLANT,169.187500
L THIGH FLUID COLLECTION,166.906944
AMC;FEVER,164.365972
S/P LIVER TX-NAUSEA-VOMITING,137.965278
APLASTIC ANEMIA;PANCYTOPENIA,137.891667
PULMONARY EMBOLISM;SUBDURAL HEMATOMA,131.970833
ACUTE MYELOGENOUS LEUKEMIA;CHEMOTHERAPY,129.960417


Load in the diagnosis data and look at the primary diagnoses.

In [17]:
diagnosis = pd.read_csv(os.path.join(path_to_download_folder, "diagnosis.csv"))

In [24]:
primary_diag = diagnosis[diagnosis['seq_num']==1]

In [26]:
primary_diag

,row_id,subject_id,hadm_id,seq_num,icd9_code,row_id.1,icd9_code.1,short_title,long_title,row_id.2,...,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
30,892,107,191941,1,99673,11632,99673,Comp-ren dialys dev/grft,Other complications due to renal dialysis devi...,114,...,Medicare,ENGL,CATHOLIC,SEPARATED,HISPANIC OR LATINO,2122-05-14 12:29:00.000,2122-05-14 20:53:00.000,UPPER GI BLEED,0,1
31,892,107,191941,1,99673,11632,99673,Comp-ren dialys dev/grft,Other complications due to renal dialysis devi...,113,...,Medicare,ENGL,CATHOLIC,SEPARATED,HISPANIC OR LATINO,2121-11-30 16:40:00.000,2121-11-30 19:57:00.000,HYPERKALEMIA;CONGESTIVE HEART FAILURE,0,1
32,892,107,191941,1,99673,11632,99673,Comp-ren dialys dev/grft,Other complications due to renal dialysis devi...,112,...,Private,ENGL,CATHOLIC,MARRIED,HISPANIC OR LATINO,NaN,NaN,AV FISTULA;ANGIOPLASTY,0,1
45,897,108,123552,1,41401,4374,41401,Crnry athrscl natve vssl,Coronary atherosclerosis of native coronary ar...,115,...,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,UNSTABLE ANGINA\CATH,0,1
51,903,109,102024,1,40301,4312,40301,Mal hyp kid w cr kid V,"Hypertensive chronic kidney disease, malignant...",149,...,Medicaid,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,2142-08-28 16:12:00.000,2142-08-28 21:15:00.000,SHORTNESS OF BREATH,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1341830,637056,96908,138027,1,53541,6103,53541,Oth spf gastrt w hmrhg,"Other specified gastritis, with hemorrhage",58009,...,Medicaid,ENGL,CATHOLIC,MARRIED,HISPANIC OR LATINO,2145-02-02 19:33:00.000,2145-02-03 00:17:00.000,UPPER GASTROINTESTINAL BLEED,0,1
1341831,637056,96908,138027,1,53541,6103,53541,Oth spf gastrt w hmrhg,"Other specified gastritis, with hemorrhage",58008,...,Medicaid,ENGL,CATHOLIC,MARRIED,HISPANIC OR LATINO,2145-01-05 09:08:00.000,2145-01-05 14:18:00.000,CONGESTIVE HEART FAILURE,0,1
1341877,637072,96908,149629,1,25040,1680,25040,DMII renl nt st uncntrld,"Diabetes with renal manifestations, type II or...",58010,...,Private,ENGL,CATHOLIC,MARRIED,HISPANIC OR LATINO,2145-09-04 05:57:00.000,2145-09-04 12:44:00.000,UPPER GI BLEED,0,1
1341878,637072,96908,149629,1,25040,1680,25040,DMII renl nt st uncntrld,"Diabetes with renal manifestations, type II or...",58009,...,Medicaid,ENGL,CATHOLIC,MARRIED,HISPANIC OR LATINO,2145-02-02 19:33:00.000,2145-02-03 00:17:00.000,UPPER GASTROINTESTINAL BLEED,0,1
